## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [52]:
%%sql
postgresql:///animal_shelters
    select * from age_costs;
    

51 rows affected.


age,costs
0,0
1,0
2,0
3,0
4,0
5,100
6,100
7,100
8,150
9,150


In [53]:
%%sql
postgresql:///animal_shelters
    select animaltype,
animal_size as size,
sum(location_cost + age_cost + size_cost) as total,
round((sum(location_cost + age_cost + size_cost))  *100.0/sum(sum(location_cost + age_cost + size_cost)) over(),2) as percentage
from (
    select
	anms.*,
	L.costs as location_cost,
	(DATE_PART('year', '2021-12-31'::date ) - DATE_PART('year', birthdate::date )) as age,
	(
	select
		costs
	from
		age_costs
	where
		age::integer = (DATE_PART('year', '2021-12-31'::date) - DATE_PART('year', birthdate::date))::integer
    ) as age_cost,
	case
		when anms.weight <= 10
		and anms.animaltype = 'Dog' then 'Small'
		when anms.weight> 10
		and anms.weight <= 30
		and anms.animaltype = 'Dog' then 'Medium'
		when anms.weight > 30
		and anms.animaltype = 'Dog' then 'Large'
		when anms.weight <= 5
		and anms.animaltype = 'Cat' then 'Small'
		when anms.weight> 5
		and anms.weight <= 7
		and anms.animaltype = 'Cat' then 'Medium'
		when anms.weight > 7
		and anms.animaltype = 'Cat' then 'Large'
		when anms.weight <= .7
		and anms.animaltype = 'Bird' then 'Small'
		when anms.weight> .7
		and anms.weight <= 1.1
		and anms.animaltype = 'Bird' then 'Medium'
		when anms.weight > 1.1
		and anms.animaltype = 'Bird' then 'Large'
		else ''
	end as animal_size,
	sc.costs as size_cost
from
	animals anms
left join location_costs L on anms.location = L.location
left join size_costs sc on
	anms.animaltype = sc.animaltype
		and sc.size =
		(case
			when anms.weight <= 10
			and anms.animaltype = 'Dog' then 'Small'
			when anms.weight> 10
			and anms.weight <= 30
			and anms.animaltype = 'Dog' then 'Medium'
			when anms.weight > 30
			and anms.animaltype = 'Dog' then 'Large'
			when anms.weight <= 5
			and anms.animaltype = 'Cat' then 'Small'
			when anms.weight> 5
			and anms.weight <= 7
			and anms.animaltype = 'Cat' then 'Medium'
			when anms.weight > 7
			and anms.animaltype = 'Cat' then 'Large'
			when anms.weight <= .7
			and anms.animaltype = 'Bird' then 'Small'
			when anms.weight> .7
			and anms.weight <= 1.1
			and anms.animaltype = 'Bird' then 'Medium'
			when anms.weight > 1.1
			and anms.animaltype = 'Bird' then 'Large'
			else ''
		end)
) data
where animalid not in (select sponsorid from sponsored_animals) 
group by animaltype , size 
order by animaltype , size desc;



9 rows affected.


animaltype,size,total,percentage
Bird,Small,1615,0.05
Bird,Medium,3460,0.10
Bird,Large,7770,0.22
Cat,Small,518015,14.90
Cat,Medium,250575,7.21
Cat,Large,439490,12.64
Dog,Small,336530,9.68
Dog,Medium,941895,27.09
Dog,Large,977665,28.12
